In [1]:
import lancedb
import pandas as pd 
import numpy as np 
import os 
import ast
from sentence_transformers import SentenceTransformer

/Users/adithisatish/TUM/Academics/SoSe24/guided_research/.env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = "../../europeancities-wikivoyage-tripadvisor/data-sources/"
wikivoyage_docs_dir = data_dir + "wikivoyage/"
wikivoyage_listings_dir = wikivoyage_docs_dir + "listings/"
airbnb_dir = data_dir + "airbnb/"

In [3]:
uri = "../database/airbnb-listings-db"
db = lancedb.connect(uri)

In [17]:
df = pd.DataFrame()
for file in os.listdir(airbnb_dir + "embeddings/"):
    emb = pd.read_csv(airbnb_dir + "embeddings/" + file)
    emb['city'] = file.split("_")[0]
    df = pd.concat([df, emb])
    print(f"concatenated {file}")

concatenated stockholm_embeddings.csv
concatenated istanbul_embeddings.csv
concatenated brussels_embeddings.csv
concatenated vienna_embeddings.csv
concatenated milan_embeddings.csv
concatenated valencia_embeddings.csv
concatenated naples_embeddings.csv
concatenated prague_embeddings.csv
concatenated copenhagen_embeddings.csv
concatenated london_embeddings.csv
concatenated rome_embeddings.csv
concatenated thessaloniki_embeddings.csv
concatenated lyon_embeddings.csv
concatenated riga_embeddings.csv
concatenated madrid_embeddings.csv
concatenated munich_embeddings.csv
concatenated berlin_embeddings.csv
concatenated porto_embeddings.csv
concatenated geneva_embeddings.csv
concatenated amsterdam_embeddings.csv
concatenated paris_embeddings.csv
concatenated bordeaux_embeddings.csv
concatenated zurich_embeddings.csv
concatenated bologna_embeddings.csv
concatenated barcelona_embeddings.csv


In [21]:
df['city']

0     stockholm
1     stockholm
2     stockholm
3     stockholm
4     stockholm
        ...    
95    barcelona
96    barcelona
97    barcelona
98    barcelona
99    barcelona
Name: city, Length: 2134, dtype: object

In [22]:
df = df[['city', 'listing_url','name', 'description', 'host_name', 'room_type', 'accommodates', 'price', 'embeddings']]

In [23]:
df.rename(columns={'embeddings':'vector'}, inplace=True)
df['vector'] = df['vector'].apply(ast.literal_eval)

In [24]:
db.drop_table("airbnb_listings", ignore_missing=True)
db.create_table("airbnb_listings", data=df)

LanceTable(connection=LanceDBConnection(/Users/adithisatish/TUM/Academics/SoSe24/guided_research/rag-sustainable-trs/vectordb/../database/airbnb-listings-db), name="airbnb_listings")

In [25]:
def embed_query(query):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    # vector_dimension = model.get_sentence_embedding_dimension()   
    embedding = model.encode(query).tolist()
    return embedding

query = "Cheap apartment in Munich"
query_embedding = embed_query(query)
table = db.open_table("airbnb_listings")
results = table.search(query_embedding).limit(5).to_list()

In [26]:
results

[{'city': 'munich',
  'listing_url': 'https://www.airbnb.com/rooms/707173026379442122',
  'name': 'Sofa-beds in beautiful apartment near City Center',
  'description': "Very centrally located, Located next to U2/U3/U6 subway, this takes you to the city center in 5 mins. Also, a 5-min walk away from Englischer Garten, 10-min walk away from Muenchner Freiheit, which has a lot of cool bars and pubs.  Public transport:Central station: 20 minsAirport: 45 minsOktoberfest ground: 15 minsNote: These would be shared areas with the host FYI. But I'm happy to help you have a great experience in your Munich stay :)",
  'host_name': 'Virang',
  'room_type': 'Shared room',
  'accommodates': 2,
  'price': '$49.00',
  'vector': [0.0649387463927269,
   -0.016899947077035904,
   0.021389933302998543,
   0.036889683455228806,
   -0.018391337245702744,
   0.11183156818151474,
   0.0356692299246788,
   0.014772610738873482,
   0.049089279025793076,
   0.08875816315412521,
   -0.04308363050222397,
   -0.106